In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
import warnings

In [1]:
!pip install xgboost

   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/99.8 MB 6.5 MB/s eta 0:00:16
   ---------------------------------------- 0.9/99.8 MB 9.8 MB/s eta 0:00:11
    --------------------------------------- 1.6/99.8 MB 11.2 MB/s eta 0:00:09
    --------------------------------------- 2.3/99.8 MB 12.1 MB/s eta 0:00:09
   - -------------------------------------- 2.9/99.8 MB 13.3 MB/s eta 0:00:08
   - -------------------------------------- 3.5/99.8 MB 13.2 MB/s eta 0:00:08
   - -------------------------------------- 4.2/99.8 MB 12.7 MB/s eta 0:00:08
   -- ------------------------------------- 5.0/99.8 MB 13.4 MB/s eta 0:00:08
   -- ------------------------------------- 5.5/99.8 MB 13.1 MB/s eta 0:00:08
   -- ------------------------------------- 6.1/99.8 MB 13.0 MB/s eta 0:00:08
   -- ------------------------------------- 6.5/99.8 MB 12.9 MB/s eta 0:00:08
   -- ------------------------------------- 7.0/99.8 MB 12.9 MB/s eta 0:00

In [6]:
df = pd.read_csv("data/train.csv")
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [7]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [36]:
categorical_value = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']
mode = df[categorical_value].mode().iloc[0]
df[categorical_value] = df[categorical_value].fillna(mode)

In [37]:
numerical_columns = ['Age', 'FoodCourt', 'RoomService', 'ShoppingMall', 'Spa', 'VRDeck']
mean = df[numerical_columns].mean()
df[numerical_columns] = df[numerical_columns].fillna(mean)

In [8]:
encoder = LabelEncoder()

In [38]:
for i in df.columns:
  if df[i].dtype == 'object':
     df[i]=encoder.fit_transform(df[i])


In [39]:
df['CryoSleep']=encoder.fit_transform(df['CryoSleep'])
df['VIP']=encoder.fit_transform(df['VIP'])
df['Transported']=encoder.fit_transform(df['Transported'])

In [40]:
df.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [62]:
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0,1,0,149,2,39.0,0,0.0,0.0,0.0,0.0,0.0,5252,0
1,1,0,0,2184,2,24.0,0,109.0,9.0,25.0,549.0,44.0,4502,1
2,2,1,0,1,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,457,0
3,3,1,0,1,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,7149,0
4,4,0,0,2186,2,16.0,0,303.0,70.0,151.0,565.0,2.0,8319,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,8688,1,0,146,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,3524,0
8689,8689,0,1,5280,1,18.0,0,0.0,0.0,0.0,0.0,0.0,4780,0
8690,8690,0,0,5285,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,3002,1
8691,8691,1,0,2131,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,1596,0


In [63]:
X = df.drop('Transported', axis = 1)
Y = df['Transported']

In [64]:
df.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [65]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [66]:
X

array([[-1.73185157,  0.32524452, -0.74173569, ..., -0.27666342,
        -0.26902263,  0.36833054],
       [-1.73145308, -0.83303104, -0.74173569, ...,  0.2115053 ,
        -0.23019432,  0.06830432],
       [-1.73105459,  0.32524452, -0.74173569, ...,  5.69428913,
        -0.22578201, -1.54983708],
       ...,
       [ 1.73105459, -0.83303104, -0.74173569, ..., -0.27577423,
        -0.26902263, -0.53174811],
       [ 1.73145308,  0.32524452, -0.74173569, ...,  0.03722284,
         2.58574044, -1.09419726],
       [ 1.73185157,  0.32524452, -0.74173569, ..., -0.27666342,
        -0.25843309,  0.81956997]])

In [67]:
has_nan=np.isnan(np.sum(X))
has_nan = np.isnan(np.min(X))
print(has_nan)

False


In [68]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

In [69]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    return mae, rmse, mse

In [74]:
models = {
    "XGboost regressor": XGBRegressor(),
    'LinearRegression' : LinearRegression(),
    "SVC" : SVC(gamma='auto')

}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, Y_train)

    Y_train_pred = model.predict(X_train)
    Y_test_pred = model.predict(X_test)

    model_train_r2 = evaluate_model(Y_train,Y_train_pred)

    model_test_r2 = evaluate_model(Y_test, Y_test_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model performance for training set")
   # print(model_train_mae)
    print(model_train_r2)

    print("----------------------------------------")

    print("Model performance for test set")
    #print(model_test_rmse)
    print(model_test_r2)




XGboost regressor
Model performance for training set
(0.13064107390138513, 0.1894473927098636, 0.03589031460456528)
----------------------------------------
Model performance for test set
(0.2840463504792422, 0.3876026652971386, 0.15023582614544564)
LinearRegression
Model performance for training set
(0.3712542275868916, 0.41967042177898367, 0.17612326291615008)
----------------------------------------
Model performance for test set
(0.37181790616350807, 0.422220700146646, 0.17827031963232395)
SVC
Model performance for training set
(0.19671322925225965, 0.4435236512884737, 0.19671322925225965)
----------------------------------------
Model performance for test set
(0.20552147239263804, 0.4533447610733337, 0.20552147239263804)


In [71]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [75]:
Y_pred = clf.predict(X_test)

In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.7944785276073619